In [1]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import os
import subprocess
import re
import pandas as pd


tmpfile = r"tmpWatchList.csv"
wlfile = r"WatchList.csv"

base_search_url = "https://www.amazon.com/"

def format_search_str(word):
    return word.replace(" ", "+")


def output_to_file(lst, filename):
    if os.path.exists(filename) == False:
        with open(filename, "w", newline="") as f:
            csvwriter = csv.writer(f)
            header = ["ASIN", "Title", "Author", "Price", "Link", "WatchList Indicator"]
            csvwriter.writerow(header)
   
    with open(filename, "a", newline="") as f:
        csvwriter = csv.writer(f)
        csvwriter.writerow(lst)
    
    
def get_url(word):
    header = {
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36"
    }
    #print(base_search_url + word)
    part_url1 = "s/ref=sr_nr_p_n_feature_browse-b_mrr_3?fst=as%3Aoff&rh=n%3A283155%2Ck%3A"
    part_url2 = "%2Cp_n_feature_browse-bin%3A618073011&keywords="
    part_url3 = "&ie=UTF8&qid=1511286087&rnid=618072011"

    resp = requests.get(base_search_url + part_url1 + word + part_url2 + word + part_url3, headers=header).text
    soup = BeautifulSoup(resp,"html.parser")
    return soup


def get_tag(soup):
    row_lst = []
    li_class=soup.find_all("li", {"class":"s-result-item celwidget "})
 
    for i in li_class:
        # Get ASIN
        asin = i.get("data-asin")
        
        # Get title
        h_tag = i.find("h2")
        if h_tag:
            title = h_tag.get("data-attribute")
     
        # Get author
        span_list = i.find_all("span",{"class":"a-size-small a-color-secondary"})
        pt = 0
        author = ""
        for pl in span_list:
            if pl.text.strip() == "by":
                pt += 1
            else:
                if pt == 1:
                    author = (author + " " + pl.text.strip()).strip()
                    if author[-3:] != "and":
                        pt = 0

        # Get price
        span_tag = i.find("span", {"class": "a-offscreen"})
        if span_tag:
            price = span_tag.text.replace("$","")
        
        # Get link
        #link = (i.find("a", {"class":"a-link-normal a-text-normal"})).get("href").encode("utf8")
        link = (i.find("a", {"class":"a-link-normal a-text-normal"})).get("href").strip()
        
        row_lst = [asin, title, author, price, link]
        output_to_file(row_lst,tmpfile)
    
    
def upd_watchlist(): 
    df = pd.read_csv(tmpfile)

    #save it to the 'new_tab' in destfile
    df.to_csv(wlfile)
    

def main(fn):
    if fn == "UpdWatchList":
        input_keyword = input("Pls enter search Author and/or Title: ")
        word = format_search_str(input_keyword)
        soup = get_url(word)
        get_tag(soup)
    
        fpath = r'C:\Program Files\Microsoft Office 15\root\office15\excel.exe' 
        if os.path.exists(tmpfile)== True:
            subprocess.Popen("%s %s" % (fpath, tmpfile))
    
        input_watchlist = input("Update watchlist? ")
        if input_watchlist.lower() == "y":
            upd_watchlist()

main("UpdWatchList")
    

    

Pls enter search Author and/or Title: ty drago
Update watchlist? y
